In [2]:
from sqlalchemy import create_engine
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func
import numpy as np

In [3]:
# database_path = "Resources/final.sqlite"
engine = create_engine("sqlite:///db/final.sqlite", echo=False)
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['final', 'sample_all', 'system_all']

In [4]:
Final = Base.classes.final
Sample = Base.classes.sample_all
System = Base.classes.system_all
session = Session(engine)

In [5]:
# Display the row's columns and data in dictionary format
# first_row = session.query(Final).first()
# first_row = session.query(Sample).first()
first_row = session.query(System).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x11412c908>,
 'Serverd_Populatiion': 50,
 'City': 'CASTRO VALLEY',
 'Index': 0,
 'Water_System_Name': 'NORRIS CANYON PROPERTY OWNERS ASSN',
 'Chemical_Iron': 'Iron (Fe)',
 'Arsenic_Sample_Count_2018': 0,
 'Lead_Sample_Count_2018': 0,
 'Chemical_Detected_Above_MCL_2013_2019': 0,
 'Service_Connections': 19,
 'Zip': '94552-9520',
 'County': 'ALAMEDA',
 'Water_System_Number': 103040,
 'Iron_Sample_Count_2018': 0,
 'Chemical_Arsenic': 'Arsenic',
 'Chemical_Lead': 'Lead (Pb)',
 'Chemical_Detected_2013_2019': 3}

In [6]:
# for analyte in session.query(Final).distinct():
#     print(analyte.Analyte)

In [7]:
# Design a query that lists the chemical finding that are above MCL for each water system
# and sort the output in descending order.
lead_recent=session.query(Final.Water_System, Final.Water_System_Name, Final.County, Final.City, Final.Analyte, func.count(Final.Analyte),Final.Sample_Year).\
    filter(Final.Violation == "Above_MCL").\
    filter(Final.Analyte == "Lead (Pb)").\
    filter(Final.Sample_Year == 2019 & 2018).\
    group_by(Final.Water_System).\
    order_by(func.count(Final.Analyte).desc()).all()
lead_recent


temps = list(np.ravel(lead_recent))
# temps

In [8]:
results = session.query(Final.Water_System, Final.Water_System_Name, Final.County, Final.City,Final.Analyte,func.count(Final.Analyte),Final.Sample_Year).\
        filter(Final.Violation == "Above_MCL" ).\
        filter(Final.Analyte == "Lead (Pb)").\
        filter(Final.Sample_Year == 2019 & 2018).\
        group_by(Final.Water_System).\
        order_by(func.count(Final.Analyte).desc()).all()

lead_datas = {}
for result in results:
        lead_datas["Water_Syetem"] = result[0]
        lead_datas["System_Name"] = result[1]
        lead_datas["County"] = result[2]
        lead_datas["City"] = result[3]
        lead_datas["Analyte"] = result[4]
        lead_datas["Lead_Count"] = result[5]
        lead_datas['Sample_Year'] = result[6]

print(lead_datas)

{'Water_Syetem': 1510004, 'System_Name': 'CASA LOMA WATER CO, INC.', 'County': 'KERN', 'City': 'BAKERSFIELD', 'Analyte': 'Lead (Pb)', 'Lead_Count': 1, 'Sample_Year': 2018}


# Create system_all.csv

In [9]:

results_all = session.query(Final.Water_System, Final.Water_System_Name, Final.County, Final.City, Final.Zip, Final.Serverd_Populatiion, Final.Service_Connections, func.count(Final.Analyte)).\
        group_by(Final.Water_System).all()
results_all


# results_copper = session.query(Final.Water_System, Final.Water_System_Name, Final.County, Final.City,Final.Analyte,func.count(Final.Analyte),Final.Sample_Year).\
#         filter(Final.Violation == "Above_MCL" ).\
#         filter(Final.Analyte == "Nitrite (as NO2)").\
#         filter(Final.Sample_Year == 2019 & 2018).\
#         group_by(Final.Water_System).\
# #         order_by(func.count(Final.Analyte).desc()).all()
# results_copper
# lead_datas = {}
# for result in results:
#         lead_datas["Water_Syetem"] = result[0]
#         lead_datas["System_Name"] = result[1]
#         lead_datas["County"] = result[2]
#         lead_datas["City"] = result[3]
#         lead_datas["Analyte"] = result[4]
#         lead_datas["Lead_Count"] = result[5]
#         lead_datas['Sample_Year'] = result[6]
        
# print(lead_datas)

[(103040,
  'NORRIS CANYON PROPERTY OWNERS ASSN',
  'ALAMEDA',
  'CASTRO VALLEY',
  '94552-9520',
  50,
  19.0,
  3),
 (103041,
  'TRAILER HAVEN MOBILE HOME PARK',
  'ALAMEDA',
  'SACRAMENTO',
  '95835-2808',
  543,
  194.0,
  6),
 (105002, 'RIVERS END MARINA', 'ALAMEDA', 'BYRON', '94514', 250, 4.0, 12),
 (105003, 'CEMEX ELIOT PLANT', 'ALAMEDA', 'FOLSOM', '95630-8714', 40, 1.0, 12),
 (105009,
  'MOUNTAIN HOUSE SCHOOL',
  'ALAMEDA',
  'RANCHO SANTA MARGARITA',
  '92688',
  50,
  1.0,
  23),
 (105010,
  'EBRPD - DEL VALLE REGIONAL PARK',
  'ALAMEDA',
  'OAKLAND',
  '94605-0381',
  10000,
  101.0,
  5),
 (105012,
  'EBRPD - SUNOL REGIONAL WILDERNESS',
  'ALAMEDA',
  'OAKLAND',
  '94605-0381',
  35,
  3.0,
  2),
 (105016, 'MOUNTAIN HOUSE BAR', 'ALAMEDA', 'TRACY', '95391-9602', 25, 1.0, 3),
 (105017,
  'RANCHO LOS MOCHOS-BSA CAMP',
  'ALAMEDA',
  'SAN LEANDRO',
  '94577-1514',
  600,
  1.0,
  1),
 (105018,
  'PURPLE ORCHID RESORT SPA',
  'ALAMEDA',
  'LIVERMORE',
  '94550-9759',
  45,
  1.0

In [10]:
#show all chemical detected times of each water system 

df_all = pd.DataFrame(results_all, columns=['Water_System_Number', 'Water_System_Name', 'County', 'City','Zip','Serverd_Populatiion','Service_Connections',"Chemical_Detected_2013-2019"])
df_all.head()


,Water_System_Number,Water_System_Name,County,City,Zip,Serverd_Populatiion,Service_Connections,Chemical_Detected_2013-2019
0,103040,NORRIS CANYON PROPERTY OWNERS ASSN,ALAMEDA,CASTRO VALLEY,94552-9520,50,19.0,3
1,103041,TRAILER HAVEN MOBILE HOME PARK,ALAMEDA,SACRAMENTO,95835-2808,543,194.0,6
2,105002,RIVERS END MARINA,ALAMEDA,BYRON,94514,250,4.0,12
3,105003,CEMEX ELIOT PLANT,ALAMEDA,FOLSOM,95630-8714,40,1.0,12
4,105009,MOUNTAIN HOUSE SCHOOL,ALAMEDA,RANCHO SANTA MARGARITA,92688,50,1.0,23


In [11]:
#query only number thats above mcl

above_mcl = session.query(Final.Water_System, func.count(Final.Analyte)).\
        filter(Final.Violation == "Above_MCL" ).\
        group_by(Final.Water_System).all()
above_mcl

[(103041, 2),
 (105003, 4),
 (105009, 4),
 (105012, 2),
 (105016, 1),
 (105018, 1),
 (105019, 2),
 (110001, 283),
 (110003, 47),
 (110008, 7),
 (110010, 959),
 (110012, 8),
 (202501, 6),
 (202504, 5),
 (202509, 2),
 (202522, 1),
 (202547, 3),
 (202549, 3),
 (202562, 2),
 (202564, 2),
 (202565, 2),
 (202566, 2),
 (210001, 11),
 (210002, 11),
 (210003, 2),
 (300016, 20),
 (300019, 5),
 (300024, 36),
 (300037, 16),
 (300053, 4),
 (300062, 10),
 (300078, 11),
 (300081, 1),
 (300524, 7),
 (301038, 8),
 (310002, 8),
 (310004, 4),
 (310005, 2),
 (310006, 18),
 (310019, 6),
 (310020, 11),
 (310021, 16),
 (400001, 16),
 (400003, 2),
 (400007, 2),
 (400008, 2),
 (400010, 1),
 (400012, 1),
 (400013, 8),
 (400014, 3),
 (400016, 8),
 (400019, 1),
 (400020, 1),
 (400026, 27),
 (400027, 3),
 (400030, 3),
 (400036, 2),
 (400040, 1),
 (400041, 1),
 (400042, 2),
 (400049, 59),
 (400051, 2),
 (400058, 4),
 (400060, 1),
 (400066, 1),
 (400073, 1),
 (400078, 1),
 (400081, 14),
 (400090, 1),
 (400123, 1),
 

In [12]:
df_above_mcl = pd.DataFrame(above_mcl, columns=['Water_System_Number', "Chemical_Detected_Above_MCL_2013-2019"])
df_above_mcl.head()

,Water_System_Number,Chemical_Detected_Above_MCL_2013-2019
0,103041,2
1,105003,4
2,105009,4
3,105012,2
4,105016,1


In [13]:
merged0 = pd.merge(df_all, df_above_mcl, on='Water_System_Number', how = "left")
merged0['Chemical_Detected_Above_MCL_2013-2019'] = merged0['Chemical_Detected_Above_MCL_2013-2019'].fillna(0)
merged0

,Water_System_Number,Water_System_Name,County,City,Zip,Serverd_Populatiion,Service_Connections,Chemical_Detected_2013-2019,Chemical_Detected_Above_MCL_2013-2019
0,103040,NORRIS CANYON PROPERTY OWNERS ASSN,ALAMEDA,CASTRO VALLEY,94552-9520,50,19.0,3,0.0
1,103041,TRAILER HAVEN MOBILE HOME PARK,ALAMEDA,SACRAMENTO,95835-2808,543,194.0,6,2.0
2,105002,RIVERS END MARINA,ALAMEDA,BYRON,94514,250,4.0,12,0.0
3,105003,CEMEX ELIOT PLANT,ALAMEDA,FOLSOM,95630-8714,40,1.0,12,4.0
4,105009,MOUNTAIN HOUSE SCHOOL,ALAMEDA,RANCHO SANTA MARGARITA,92688,50,1.0,23,4.0
5,105010,EBRPD - DEL VALLE REGIONAL PARK,ALAMEDA,OAKLAND,94605-0381,10000,101.0,5,0.0
6,105012,EBRPD - SUNOL REGIONAL WILDERNESS,ALAMEDA,OAKLAND,94605-0381,35,3.0,2,2.0
7,105016,MOUNTAIN HOUSE BAR,ALAMEDA,TRACY,95391-9602,25,1.0,3,1.0
8,105017,RANCHO LOS MOCHOS-BSA CAMP,ALAMEDA,SAN LEANDRO,94577-1514,600,1.0,1,0.0
9,105018,PURPLE ORCHID RESORT SPA,ALAMEDA,LIVERMORE,94550-9759,45,1.0,2,1.0


In [14]:
results_lead = session.query(Final.Water_System, Final.Analyte,func.count(Final.Analyte)).\
        filter(Final.Violation == "Above_MCL" ).\
        filter(Final.Analyte == "Lead (Pb)").\
        filter(Final.Sample_Year == 2019 & 2018).\
        group_by(Final.Water_System).all()
#create lead_2018 dataframe 
df_lead_2018 = pd.DataFrame(results_lead, columns=['Water_System_Number', "Chemical_Lead" ,"Lead_Sample_Count_2018"])
df_lead_2018.head()

,Water_System_Number,Chemical_Lead,Lead_Sample_Count_2018
0,300024,Lead (Pb),1
1,400016,Lead (Pb),1
2,1000001,Lead (Pb),1
3,1000018,Lead (Pb),1
4,1000040,Lead (Pb),1


In [15]:
merged1 = pd.merge(merged0, df_lead_2018, on='Water_System_Number', how = "left")

merged1['Chemical_Lead'] = merged1['Chemical_Lead'].fillna("Lead (Pb)")
merged1['Lead_Sample_Count_2018'] = merged1['Lead_Sample_Count_2018'].fillna(0)
merged1

,Water_System_Number,Water_System_Name,County,City,Zip,Serverd_Populatiion,Service_Connections,Chemical_Detected_2013-2019,Chemical_Detected_Above_MCL_2013-2019,Chemical_Lead,Lead_Sample_Count_2018
0,103040,NORRIS CANYON PROPERTY OWNERS ASSN,ALAMEDA,CASTRO VALLEY,94552-9520,50,19.0,3,0.0,Lead (Pb),0.0
1,103041,TRAILER HAVEN MOBILE HOME PARK,ALAMEDA,SACRAMENTO,95835-2808,543,194.0,6,2.0,Lead (Pb),0.0
2,105002,RIVERS END MARINA,ALAMEDA,BYRON,94514,250,4.0,12,0.0,Lead (Pb),0.0
3,105003,CEMEX ELIOT PLANT,ALAMEDA,FOLSOM,95630-8714,40,1.0,12,4.0,Lead (Pb),0.0
4,105009,MOUNTAIN HOUSE SCHOOL,ALAMEDA,RANCHO SANTA MARGARITA,92688,50,1.0,23,4.0,Lead (Pb),0.0
5,105010,EBRPD - DEL VALLE REGIONAL PARK,ALAMEDA,OAKLAND,94605-0381,10000,101.0,5,0.0,Lead (Pb),0.0
6,105012,EBRPD - SUNOL REGIONAL WILDERNESS,ALAMEDA,OAKLAND,94605-0381,35,3.0,2,2.0,Lead (Pb),0.0
7,105016,MOUNTAIN HOUSE BAR,ALAMEDA,TRACY,95391-9602,25,1.0,3,1.0,Lead (Pb),0.0
8,105017,RANCHO LOS MOCHOS-BSA CAMP,ALAMEDA,SAN LEANDRO,94577-1514,600,1.0,1,0.0,Lead (Pb),0.0
9,105018,PURPLE ORCHID RESORT SPA,ALAMEDA,LIVERMORE,94550-9759,45,1.0,2,1.0,Lead (Pb),0.0


In [16]:
#results_Arsenic_2018 
results_Arsenic_2018 = session.query(Final.Water_System, Final.Analyte,func.count(Final.Analyte)).\
        filter(Final.Violation == "Above_MCL" ).\
        filter(Final.Analyte == "Arsenic").\
        filter(Final.Sample_Year == 2019 & 2018).\
        group_by(Final.Water_System).all()
results_Arsenic_2018 = pd.DataFrame(results_Arsenic_2018, columns=['Water_System_Number', "Chemical_Arsenic" ,"Arsenic_Sample_Count_2018"])
results_Arsenic_2018.head()

,Water_System_Number,Chemical_Arsenic,Arsenic_Sample_Count_2018
0,300524,Arsenic,1
1,400049,Arsenic,3
2,600008,Arsenic,4
3,707507,Arsenic,2
4,707556,Arsenic,1


In [17]:
merged2 = pd.merge(merged1, results_Arsenic_2018, on='Water_System_Number', how = "left")
# merged2
merged2['Chemical_Arsenic'] = merged2['Chemical_Arsenic'].fillna("Arsenic")
merged2['Arsenic_Sample_Count_2018'] = merged2['Arsenic_Sample_Count_2018'].fillna(0)
merged2

,Water_System_Number,Water_System_Name,County,City,Zip,Serverd_Populatiion,Service_Connections,Chemical_Detected_2013-2019,Chemical_Detected_Above_MCL_2013-2019,Chemical_Lead,Lead_Sample_Count_2018,Chemical_Arsenic,Arsenic_Sample_Count_2018
0,103040,NORRIS CANYON PROPERTY OWNERS ASSN,ALAMEDA,CASTRO VALLEY,94552-9520,50,19.0,3,0.0,Lead (Pb),0.0,Arsenic,0.0
1,103041,TRAILER HAVEN MOBILE HOME PARK,ALAMEDA,SACRAMENTO,95835-2808,543,194.0,6,2.0,Lead (Pb),0.0,Arsenic,0.0
2,105002,RIVERS END MARINA,ALAMEDA,BYRON,94514,250,4.0,12,0.0,Lead (Pb),0.0,Arsenic,0.0
3,105003,CEMEX ELIOT PLANT,ALAMEDA,FOLSOM,95630-8714,40,1.0,12,4.0,Lead (Pb),0.0,Arsenic,0.0
4,105009,MOUNTAIN HOUSE SCHOOL,ALAMEDA,RANCHO SANTA MARGARITA,92688,50,1.0,23,4.0,Lead (Pb),0.0,Arsenic,0.0
5,105010,EBRPD - DEL VALLE REGIONAL PARK,ALAMEDA,OAKLAND,94605-0381,10000,101.0,5,0.0,Lead (Pb),0.0,Arsenic,0.0
6,105012,EBRPD - SUNOL REGIONAL WILDERNESS,ALAMEDA,OAKLAND,94605-0381,35,3.0,2,2.0,Lead (Pb),0.0,Arsenic,0.0
7,105016,MOUNTAIN HOUSE BAR,ALAMEDA,TRACY,95391-9602,25,1.0,3,1.0,Lead (Pb),0.0,Arsenic,0.0
8,105017,RANCHO LOS MOCHOS-BSA CAMP,ALAMEDA,SAN LEANDRO,94577-1514,600,1.0,1,0.0,Lead (Pb),0.0,Arsenic,0.0
9,105018,PURPLE ORCHID RESORT SPA,ALAMEDA,LIVERMORE,94550-9759,45,1.0,2,1.0,Lead (Pb),0.0,Arsenic,0.0


In [18]:
#results_Arsenic_2018 
results_Iron_2018 = session.query(Final.Water_System, Final.Analyte,func.count(Final.Analyte)).\
        filter(Final.Violation == "Above_MCL" ).\
        filter(Final.Analyte == "Iron (Fe)").\
        filter(Final.Sample_Year == 2019 & 2018).\
        group_by(Final.Water_System).all()
results_Iron_2018 = pd.DataFrame(results_Iron_2018, columns=['Water_System_Number', "Chemical_Iron" ,"Iron_Sample_Count_2018"])
results_Iron_2018.head()

,Water_System_Number,Chemical_Iron,Iron_Sample_Count_2018
0,105019,Iron (Fe),1
1,110001,Iron (Fe),1
2,110008,Iron (Fe),1
3,110010,Iron (Fe),8
4,210002,Iron (Fe),1


In [19]:
merged3 = pd.merge(merged2, results_Iron_2018, on='Water_System_Number', how = "left")

merged3['Chemical_Iron'] = merged3['Chemical_Iron'].fillna("Iron (Fe)")
merged3
merged3['Iron_Sample_Count_2018'] = merged3['Iron_Sample_Count_2018'].fillna(0)
merged3



,Water_System_Number,Water_System_Name,County,City,Zip,Serverd_Populatiion,Service_Connections,Chemical_Detected_2013-2019,Chemical_Detected_Above_MCL_2013-2019,Chemical_Lead,Lead_Sample_Count_2018,Chemical_Arsenic,Arsenic_Sample_Count_2018,Chemical_Iron,Iron_Sample_Count_2018
0,103040,NORRIS CANYON PROPERTY OWNERS ASSN,ALAMEDA,CASTRO VALLEY,94552-9520,50,19.0,3,0.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
1,103041,TRAILER HAVEN MOBILE HOME PARK,ALAMEDA,SACRAMENTO,95835-2808,543,194.0,6,2.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
2,105002,RIVERS END MARINA,ALAMEDA,BYRON,94514,250,4.0,12,0.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
3,105003,CEMEX ELIOT PLANT,ALAMEDA,FOLSOM,95630-8714,40,1.0,12,4.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
4,105009,MOUNTAIN HOUSE SCHOOL,ALAMEDA,RANCHO SANTA MARGARITA,92688,50,1.0,23,4.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
5,105010,EBRPD - DEL VALLE REGIONAL PARK,ALAMEDA,OAKLAND,94605-0381,10000,101.0,5,0.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
6,105012,EBRPD - SUNOL REGIONAL WILDERNESS,ALAMEDA,OAKLAND,94605-0381,35,3.0,2,2.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
7,105016,MOUNTAIN HOUSE BAR,ALAMEDA,TRACY,95391-9602,25,1.0,3,1.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
8,105017,RANCHO LOS MOCHOS-BSA CAMP,ALAMEDA,SAN LEANDRO,94577-1514,600,1.0,1,0.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0
9,105018,PURPLE ORCHID RESORT SPA,ALAMEDA,LIVERMORE,94550-9759,45,1.0,2,1.0,Lead (Pb),0.0,Arsenic,0.0,Iron (Fe),0.0


In [20]:
for analyte in session.query(Sample).distinct():
    print(analyte.ANALYTE)

Source Temperature C
Color, Apparent (Unfiltered)
Odor Threshold @ 60 C
Specific Conductance (E.C.)
Field pH
PH, Field
PH, Laboratory
Alkalinity, (Total) (as CaCO3 equivalents)
Bicarbonate Alkalinity (as HCO3)
Carbonate Alkalinity (as CO3)
Ammonia (NH3-N)
Nitrate as Nitrogen (N)
Nitrite as Nitrogen (N)
Phosphate (as PO4)
Phosphate, Ortho (as PO4)
Total Organic Carbon (TOC)
Sulfide
Hardness, (Total) as CACO3
Calcium (Ca)
Magnesium (Mg)
Sodium (Na)
Sodium Absorption  Ratio
Potassium (K)
Chloride
Sulfate (SO4)
Fluoride (F) (Natural-Source)
Silica
Arsenic
Barium (Ba)
Beryllium
Boron
Cadmium (Cd)
Chromium, hexavalent (CrVI)
Chromium (Total Cr)
Cobalt
Copper (Cu)
Iron (Fe)
Iron, Dissolved
Lead (Pb)
Manganese (Mn)
Manganese, Dissolved
Thallium
Molybdenum
Nickel
Silver (Ag)
Vanadium
Zinc (Zn)
Antimony
Aluminum (Al)
Lithium
Selenium (Se)
Cyanide
Gross Alpha
Gross Alpha Counting Error
Gross Beta
Gross Beta Counting Error
Acetochlor
Tritium
Tritium Counting Error
Radium 226
Radium 226 Counting Er

In [21]:
merged3.to_csv('system_all.csv')

# 1. First Route/watersystem

In [22]:
# database_path = "Resources/final.sqlite"
engine = create_engine("sqlite:///db/final.sqlite", echo=False)
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['final', 'sample_all', 'system_all']

In [23]:
Final = Base.classes.final
Sample = Base.classes.sample_all
System = Base.classes.system_all
session = Session(engine)

In [24]:
#get column names of the system table 

In [25]:
# List all of the water system found in the final table
# session.query(Final.Water_System).group_by(Final.Water_System).all()

water_system = session.query(System.Water_System_Number).all
water_system

<bound method Query.all of <sqlalchemy.orm.query.Query object at 0x1142f4780>>

In [26]:
# first route query all water system 
    # Use Pandas to perform the sql query
stmt = session.query(Sample).statement
df = pd.read_sql_query(stmt, session.bind)
df.columns[3:]

Index(['103039', '103040', '103041', '104010', '105002', '105003', '105008',
       '105009', '105010', '105012',
       ...
       '1504005', '1504006', '1504007', '1504008', '1504009', '1510001',
       '1510002', '1510003', '1510004', '1510005'],
      dtype='object', length=1663)

In [27]:
#second rout query info of each water system 
#select comlumns 

# def system_all(Water_System_Number):
    
sel = [
        System.Water_System_Number,
        System.Water_System_Name,
        System.County,
        System.City,
        System.Zip,
        System.Serverd_Populatiion,
        System.Service_Connections,
        System.Chemical_Detected_2013_2019,
        System.Chemical_Detected_Above_MCL_2013_2019,
        System.Lead_Sample_Count_2018,
        System.Arsenic_Sample_Count_2018,
        System.Iron_Sample_Count_2018]

#     results = session.query(*sel).filter(System.Water_System_Number == Water_System_Number).all()
results_system = session.query(*sel).all()
results_system

    # Create a dictionary entry for each row of system information
system_all = {} 
for result in results_system:
        system_all["CA_Water_System_Number"] = result[0]
        system_all["Water_System Name"] = result[1]
        system_all["County"] = result[2]
        system_all["City"] = result[3]
        system_all["Zip_Code"] = result[4]
        system_all["Served_Population"] = result[5]
        system_all["Service_Connection"] = result[6]
        system_all["Total_Numeber_of_Chemical_Detected(2013_2019)"] = result[7]
        system_all["Total_Numeber_of_Chemical_Detected_above_MCL(2013_2019)"] = result[8]
        system_all["Total_Number_of_Finding_of_Lead_above_MCL(2018_2019)"] = result[9]
        system_all["Total_Number_of_Finding_of_Arsenic_above_MCL(2018_2019)"] = result[10]
        system_all["Total_Number_of_Finding_of_Iron_above_MCL(2018_2019)"] = result[11]

system_all


{'CA_Water_System_Number': 1510005,
 'Water_System Name': 'DELANO, CITY OF',
 'County': 'KERN',
 'City': 'DELANO',
 'Zip_Code': '93215',
 'Served_Population': 56623,
 'Service_Connection': 9544,
 'Total_Numeber_of_Chemical_Detected(2013_2019)': 2888,
 'Total_Numeber_of_Chemical_Detected_above_MCL(2013_2019)': 1702,
 'Total_Number_of_Finding_of_Lead_above_MCL(2018_2019)': 0,
 'Total_Number_of_Finding_of_Arsenic_above_MCL(2018_2019)': 270,
 'Total_Number_of_Finding_of_Iron_above_MCL(2018_2019)': 0}

In [28]:
stmt = session.query(Sample).statement
df = pd.read_sql_query(stmt, session.bind)

,Index,STORE_NUM,ANALYTE,103039,103040,103041,104010,105002,105003,105008,...,1504005,1504006,1504007,1504008,1504009,1510001,1510002,1510003,1510004,1510005
0,0,10,Source Temperature C,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1028,0,0
1,1,81,"Color, Apparent (Unfiltered)",1,0,2,0,0,2,0,...,1,2,1,1,1,12,2,260,6,26
2,2,86,Odor Threshold @ 60 C,1,0,2,0,0,2,0,...,1,2,1,1,1,12,2,263,6,26
3,3,95,Specific Conductance (E.C.),1,0,2,0,0,3,0,...,1,3,1,1,1,15,3,193,6,264
4,4,400,Field pH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1932,0,0
5,5,400,"PH, Field",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1932,0,0
6,6,403,"PH, Laboratory",1,0,2,0,0,2,0,...,1,2,1,1,1,12,2,317,6,26
7,7,410,"Alkalinity, (Total) (as CaCO3 equivalents)",0,0,2,0,0,2,0,...,1,2,1,1,1,12,2,349,6,26
8,8,440,Bicarbonate Alkalinity (as HCO3),0,0,2,0,0,2,0,...,1,2,1,1,1,12,2,351,6,26
9,9,445,Carbonate Alkalinity (as CO3),0,0,2,0,0,2,0,...,1,2,1,1,1,12,2,351,6,26
